In [ ]:
import pickle
fpaths = [
# '/group/jug/ashesh/data/paper_stats/Test_P64_G32_M50_Sk8/calib_stats_training_disentangle_2404_D21-M3-S0-L8_6.pkl',
# '/group/jug/ashesh/data/paper_stats/Test_P64_G32_M50_Sk8/calib_stats_training_disentangle_2404_D25-M3-S0-L8_143.pkl',
# '/group/jug/ashesh/data/paper_stats/Test_P64_G32_M50_Sk8/calib_stats_training_disentangle_2404_D25-M3-S0-L8_111.pkl',
# '/group/jug/ashesh/data/paper_stats/Test_P64_G32_M50_Sk8/calib_stats_training_disentangle_2404_D25-M3-S0-L8_97.pkl',
# '/group/jug/ashesh/data/paper_stats/Test_P64_G32_M50_Sk8/calib_stats_training_disentangle_2404_D25-M3-S0-L8_120.pkl',
# '/group/jug/ashesh/data/paper_stats/Test_P64_G32_M50_Sk8/calib_stats_training_disentangle_2405_D25-M3-S0-L8_2.pkl',
# '/group/jug/ashesh/data/paper_stats/Test_P64_G32_M50_Sk8/calib_stats_training_disentangle_2404_D25-M3-S0-L8_125.pkl',
# '/group/jug/ashesh/data/paper_stats/Test_P64_G32_M50_Sk8/calib_stats_training_disentangle_2405_D25-M3-S0-L8_3.pkl',
# '/group/jug/ashesh/data/paper_stats/Test_P64_G32_M50_Sk8/calib_stats_training_disentangle_2404_D25-M3-S0-L8_139.pkl',
# '/group/jug/ashesh/data/paper_stats/Test_P64_G32_M50_Sk0/calib_stats_training_disentangle_2405_D18-M3-S0-L8_14.pkl',
# '/group/jug/ashesh/data/paper_stats/Test_P64_G32_M50_Sk8/calib_stats_training_disentangle_2406_D25-M3-S0-L8_4.pkl',
# '/group/jug/ashesh/data/paper_stats/Test_P64_G32_M50_Sk8/calib_stats_training_disentangle_2406_D25-M3-S0-L8_12.pkl'
# '/group/jug/ashesh/data/paper_stats/Test_P64_G32_M50_Sk0/calib_stats_training_disentangle_2406_D25-M3-S0-L8_4.pkl'
# '/group/jug/ashesh/data/paper_stats/Test_P64_G64_M3_Sk0/calib_stats_training_disentangle_2406_D25-M3-S0-L8_4.pkl'
# '/group/jug/ashesh/data/paper_stats/Val_P64_G64_M3_Sk0/calib_stats_training_disentangle_2406_D25-M3-S0-L8_4.pkl',
'/group/jug/ashesh/data/paper_stats/Test_P64_G32_M50_Sk0/calib_stats_training_disentangle_2406_D25-M3-S0-L8_4.pkl'

]
calib_fpath =fpaths[-1]
with open(calib_fpath, 'rb') as f:
    calib_stats = pickle.load(f)


In [ ]:
from disentangle.analysis.paper_plots import get_first_index, get_last_index
q_s = 0.00001
q_e = 0.99999


In [ ]:
import matplotlib.pyplot as plt
import os

_,ax = plt.subplots(figsize=(2,2))
for ch_idx in range(len(calib_stats['calib_stats'])):
    tmp_stats = calib_stats['calib_stats'][ch_idx]
    tmp_rmv = tmp_stats['rmv']
    tmp_rmse = tmp_stats['rmse']
    count = tmp_stats['bin_count']
    first_idx = get_first_index(count, q_s)
    last_idx = get_last_index(count, q_e)
    plt.plot(tmp_rmv[first_idx:-last_idx],
            tmp_rmse[first_idx:-last_idx],
            '-+',
            label='C{}'.format(ch_idx)
            )
plt.tight_layout()
plt.legend()


In [ ]:
import numpy as np
def plot_calib(ax, calib_stats, ch_idx, color='b'):
    tmp_stats = calib_stats['calib_stats'][ch_idx]
    tmp_rmv = tmp_stats['rmv']
    tmp_rmse = tmp_stats['rmse']
    count = tmp_stats['bin_count']
    rmse_err = tmp_stats['rmse_err']

    first_idx = get_first_index(count, q_s)
    last_idx = get_last_index(count, q_e)
    # plot the calibration curve with error bars     
    ax.plot(tmp_rmv[first_idx:-last_idx],
            tmp_rmse[first_idx:-last_idx],
            '-+',
            label='C{}'.format(ch_idx),
            color=color,
            )
    rmse_floor = np.array(tmp_rmse[first_idx:-last_idx]) - np.array(rmse_err[first_idx:-last_idx])
    rmse_ceil = np.array(tmp_rmse[first_idx:-last_idx]) + np.array(rmse_err[first_idx:-last_idx])
    ax.fill_between(tmp_rmv[first_idx:-last_idx], rmse_floor, rmse_ceil, alpha=0.3, label='error band')

    # enable the grid and set the background color to gray 
    ax.grid(True)
    ax.set_facecolor('0.75')
    # get x and y limits of the plot
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    minv = min(xlim[0], ylim[0])
    maxv = max(xlim[1], ylim[1])
    ax.plot([minv, maxv], [minv, maxv], '--', color='black')
    print('xlim:', xlim)
    print('ylim:', ylim)     



In [ ]:
_,ax = plt.subplots(figsize=(3,8),nrows=3)
plot_calib(ax[0],calib_stats, 0, 'b')
plot_calib(ax[1],calib_stats, 1, 'b')
plot_calib(ax[2],calib_stats, 2, 'b')
for i in range(3):
    ax[i].yaxis.set_label_position("right")
    ax[i].yaxis.tick_right()

fname = os.path.basename(calib_fpath).replace('.pkl','.png')
fpath = os.path.join('/group/jug/ashesh/naturemethods/calibration', fname)
plt.tight_layout()
plt.savefig(fpath, dpi=200)
print('Saved to:', fpath)
